In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, f1_score


In [11]:
import pandas as pd

# Load your tickets dataset
tickets = pd.read_csv("D:\Workspace\Projects\CTS\Dataset\customer_support_tickets.csv")  # or pd.read_excel("tickets.xlsx")
tickets.head()


,Ticket ID,Customer Name,Customer Email,Customer Age,Customer Gender,Product Purchased,Date of Purchase,Ticket Type,Ticket Subject,Ticket Description,Ticket Status,Resolution,Ticket Priority,Ticket Channel,First Response Time,Time to Resolution,Customer Satisfaction Rating
0,1,Marisa Obrien,carrollallison@example.com,32,Other,GoPro Hero,2021-03-22,Technical issue,Product setup,I'm having an issue with the {product_purchase...,Pending Customer Response,NaN,Critical,Social media,2023-06-01 12:15:36,NaN,NaN
1,2,Jessica Rios,clarkeashley@example.com,42,Female,LG Smart TV,2021-05-22,Technical issue,Peripheral compatibility,I'm having an issue with the {product_purchase...,Pending Customer Response,NaN,Critical,Chat,2023-06-01 16:45:38,NaN,NaN
2,3,Christopher Robbins,gonzalestracy@example.com,48,Other,Dell XPS,2020-07-14,Technical issue,Network problem,I'm facing a problem with my {product_purchase...,Closed,Case maybe show recently my computer follow.,Low,Social media,2023-06-01 11:14:38,2023-06-01 18:05:38,3.0
3,4,Christina Dillon,bradleyolson@example.org,27,Female,Microsoft Office,2020-11-13,Billing inquiry,Account access,I'm having an issue with the {product_purchase...,Closed,Try capital clearly never color toward story.,Low,Social media,2023-06-01 07:29:40,2023-06-01 01:57:40,3.0
4,5,Alexander Carroll,bradleymark@example.com,67,Female,Autodesk AutoCAD,2020-02-04,Billing inquiry,Data loss,I'm having an issue with the {product_purchase...,Closed,West decision evidence bit.,Low,Email,2023-06-01 00:12:42,2023-06-01 19:53:42,1.0


In [12]:
# Convert numeric columns safely
tickets["Customer Satisfaction Rating"] = pd.to_numeric(
    tickets.get("Customer Satisfaction Rating", pd.Series([0]*len(tickets))),
    errors="coerce"
).fillna(0)

# Ensure 'Product Purchased' exists
if 'Product Purchased' not in tickets.columns:
    tickets['Product Purchased'] = 'Unknown'


In [14]:
# If 'Upsell_Label' exists, keep it; otherwise create a column of zeros
if 'Upsell_Label' not in tickets.columns:
    tickets['Upsell_Label'] = 0
else:
    tickets['Upsell_Label'] = tickets['Upsell_Label'].astype(int)


In [15]:
from sklearn.model_selection import train_test_split

X = tickets.drop(columns=['Upsell_Label', 'Product Purchased'])
y = tickets['Upsell_Label']

X_train_u, X_test_u, y_train_u, y_test_u = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [17]:
from sklearn.preprocessing import LabelEncoder

# Copy the dataset for upsell model
upsell_data = tickets.copy()

# Drop columns that won't be used
cols_to_drop = ['Customer Name', 'Customer Email', 'Ticket Subject', 'Ticket Description',
                'Resolution']  # Keep features that are meaningful
upsell_data.drop(columns=cols_to_drop, inplace=True, errors='ignore')

# Convert categorical columns to numeric
cat_cols = upsell_data.select_dtypes(include='object').columns
for col in cat_cols:
    upsell_data[col] = LabelEncoder().fit_transform(upsell_data[col].astype(str))

# Convert datetime columns to timestamp
date_cols = ['Date of Purchase', 'First Response Time', 'Time to Resolution']
for col in date_cols:
    if col in upsell_data.columns:
        upsell_data[col] = pd.to_datetime(upsell_data[col], errors='coerce').astype('int64') // 10**9
        upsell_data[col].fillna(0, inplace=True)

# Features and target
X_train_u = upsell_data.drop(columns=['Upsell_Label'], errors='ignore')
y_train_u = upsell_data['Upsell_Label']

# Train XGBoost
from xgboost import XGBClassifier
upsell_model = XGBClassifier(
    n_estimators=200,
    max_depth=6,
    learning_rate=0.1,
    objective='binary:logistic',
    random_state=42,
    enable_categorical=True
)
upsell_model.fit(X_train_u, y_train_u)
print("Upsell XGBoost model trained successfully!")


Upsell XGBoost model trained successfully!


C:\Users\samsd\AppData\Local\Temp\ipykernel_17804\3086814231.py:21: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  upsell_data[col].fillna(0, inplace=True)
C:\Users\samsd\AppData\Local\Temp\ipykernel_17804\3086814231.py:21: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, 

In [18]:
# Convert datetime columns to numeric timestamp
for col in tickets.select_dtypes(include='datetime64').columns:
    tickets[col] = tickets[col].astype('int64') // 10**9

# Identify categorical columns
cat_cols = tickets.select_dtypes(include='object').columns.tolist()
num_cols = tickets.select_dtypes(include=['int64', 'float64']).columns.tolist()


In [19]:
X_prod = tickets.drop(columns=['Product Purchased', 'Upsell_Label'])
y_prod = tickets['Product Purchased']

X_train_p, X_test_p, y_train_p, y_test_p = train_test_split(
    X_prod, y_prod, test_size=0.2, random_state=42, stratify=y_prod
)


In [32]:
# ==========================
# 1️⃣ Preprocessing
# ==========================
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from sklearn.preprocessing import LabelEncoder

# Assume 'tickets' dataframe is already loaded
# Fill missing values
tickets.fillna({
    'Product Purchased': 'Unknown',
    'Customer Satisfaction Rating': 0
}, inplace=True)

# Convert datetime columns to numeric
datetime_cols = ['Date of Purchase', 'First Response Time']
for col in datetime_cols:
    if col in tickets.columns:
        tickets[col] = pd.to_datetime(tickets[col], errors='coerce')
        tickets[col] = tickets[col].fillna(pd.Timestamp('1970-01-01'))
        tickets[col] = tickets[col].view(np.int64) // 10**9  # seconds since epoch

# Encode target variable
tickets['Product_Label'] = LabelEncoder().fit_transform(tickets['Product Purchased'])

# Identify categorical features (string/object)
cat_cols = tickets.select_dtypes(include='object').columns.tolist()

# Before dropping target columns for CatBoost
tickets_original = tickets.copy()  # keep a full copy

# Drop target column from features
features = tickets.drop(columns=['Product Purchased', 'Product_Label'])

# Train-test split
X_train_p, X_test_p, y_train_p, y_test_p = train_test_split(
    features, tickets['Product_Label'], test_size=0.2, random_state=42
)

# Get column indices for CatBoost categorical features
cat_idx = [X_train_p.columns.get_loc(c) for c in cat_cols if c in X_train_p.columns]

# ==========================
# 2️⃣ Train CatBoost Recommender
# ==========================
recommender = CatBoostClassifier(
    iterations=300,
    depth=6,
    learning_rate=0.1,
    loss_function='MultiClass',
    random_state=42,
    verbose=100
)

recommender.fit(X_train_p, y_train_p, cat_features=cat_idx)

# ==========================
# 3️⃣ Top-K Evaluation
# ==========================
K = 3
preds_proba = recommender.predict_proba(X_test_p)

# Get Top-K predicted class indices
top_k_preds = np.argsort(preds_proba, axis=1)[:, -K:][:, ::-1]

# Map indices back to product names
product_mapping = dict(enumerate(LabelEncoder().fit(tickets['Product Purchased']).classes_))
top_k_products = [[product_mapping[idx] for idx in row] for row in top_k_preds]

# Show first 5 examples
for i in range(5):
    print(f"Actual: {tickets['Product Purchased'].iloc[X_test_p.index[i]]}, Top-{K} Predictions: {top_k_products[i]}")


C:\Users\samsd\AppData\Local\Temp\ipykernel_17804\3700261146.py:23: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  tickets[col] = tickets[col].view(np.int64) // 10**9  # seconds since epoch
C:\Users\samsd\AppData\Local\Temp\ipykernel_17804\3700261146.py:23: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  tickets[col] = tickets[col].view(np.int64) // 10**9  # seconds since epoch


0:	learn: 3.7364948	total: 1.9s	remaining: 9m 26s
100:	learn: 3.4748663	total: 3m 34s	remaining: 7m 1s
200:	learn: 3.1563149	total: 10m 8s	remaining: 4m 59s
299:	learn: 2.8485143	total: 19m 4s	remaining: 0us
Actual: Roomba Robot Vacuum, Top-3 Predictions: ['Sony Xperia', 'Microsoft Surface', 'Garmin Forerunner']
Actual: Roomba Robot Vacuum, Top-3 Predictions: ['Sony Xperia', 'Bose QuietComfort', 'GoPro Hero']
Actual: Philips Hue Lights, Top-3 Predictions: ['GoPro Hero', 'Garmin Forerunner', 'Microsoft Xbox Controller']
Actual: LG OLED, Top-3 Predictions: ['Nest Thermostat', 'Sony Xperia', 'Garmin Forerunner']
Actual: Roomba Robot Vacuum, Top-3 Predictions: ['Sony Xperia', 'Canon EOS', 'Garmin Forerunner']


In [39]:
# ==========================
# Cell 11: Model Evaluation
# ==========================

from sklearn.metrics import accuracy_score, f1_score
import numpy as np

# --------------------------
# 1️⃣ Upsell Model Evaluation
# --------------------------

# Restore 'Product Purchased' for Upsell evaluation
X_train_u['Product Purchased'] = tickets_original.loc[X_train_u.index, 'Product Purchased']
X_test_u['Product Purchased'] = tickets_original.loc[X_test_u.index, 'Product Purchased']

# Ensure categorical types for XGBoost
for col in X_train_u.select_dtypes(include='object').columns:
    X_train_u[col] = X_train_u[col].astype('category')
for col in X_test_u.select_dtypes(include='object').columns:
    X_test_u[col] = X_test_u[col].astype('category')

# Ensure same columns order
X_test_u_fixed = X_test_u[X_train_u.columns]

# Predict
y_pred_u = upsell_model.predict(X_test_u_fixed)

# Metrics
accuracy_u = accuracy_score(y_test_u, y_pred_u)
f1_u = f1_score(y_test_u, y_pred_u, average='weighted')
loss_percentage_u = 100 - (accuracy_u * 100)

print("🔹 Upsell Model Metrics")
print(f"Accuracy: {accuracy_u:.4f}")
print(f"F1 Score: {f1_u:.4f}")
print(f"Loss %: {loss_percentage_u:.2f}%\n")


# ------------------------------
# 2️⃣ Recommendation Model Eval
# ------------------------------

# Ensure categorical columns for CatBoost
X_test_p_fixed = X_test_p.copy()
for col in X_test_p_fixed.select_dtypes(include='object').columns:
    X_test_p_fixed[col] = X_test_p_fixed[col].astype('category')

# Predict
y_pred_r = recommender.predict(X_test_p_fixed)

# Metrics
accuracy_r = accuracy_score(y_test_p, y_pred_r)
f1_r = f1_score(y_test_p, y_pred_r, average='weighted')
loss_percentage_r = 100 - (accuracy_r * 100)

print("🔹 Recommendation Model Metrics")
print(f"Accuracy: {accuracy_r:.4f}")
print(f"F1 Score: {f1_r:.4f}")
print(f"Loss %: {loss_percentage_r:.2f}%")


🔹 Upsell Model Metrics
Accuracy: 1.0000
F1 Score: 1.0000
Loss %: 0.00%

🔹 Recommendation Model Metrics
Accuracy: 0.0195
F1 Score: 0.0169
Loss %: 98.05%
